# Projeto: Análise de Anomalias em Série Temporal

BIMASTER - Trabalho de final de curso

Nome: Alex Marques Campos

Etapa 02.A: Detecção de anomalias via métodos estatísticos

O objetivo deste notebook é carregar os dados das séries históricas de interesse e observar se é possível utilizar métodos estatísticos para realizar a detecção de anomalias nos dados. Para isso, focaremos no uso do __z-score modificado__.

O primeiro passo é carregar as bibliotecas necessárias ao processamento e os dados propriamente ditos, que estão armazenados nos arquivos CSV (_comma separated values_) armazenados no subdiretório './__dados__/'.

## Configuração do ambiente de execução

In [1]:
# Garantimos que a versão do statsmodels está fixa com a versão que precisamos
# para a análise.
!pip install statsmodels==0.13.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
#import os
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

from functools import partial
from pathlib import Path
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

In [3]:
NOME_DIRETORIO_DADOS = 'dados'
NOME_ARQUIVO_SERIE_IBCBR = 'serie_ibcbr.csv'
NOME_ARQUIVO_SERIE_IBCBR_RESIDUO = 'serie_ibcbr_residuo.csv'
NOME_ARQUIVO_SERIE_IBCBR_DIFERENCAS = 'serie_ibcbr_diferencas.csv'
NOME_ARQUIVO_SERIE_IBCBR_DIFERENCAS_RESIDUO = 'serie_ibcbr_diferencas_residuo.csv'

In [4]:
print(f'pandas == {pd.__version__}')
print(f'statsmodels == {sm.__version__}')
print(f'matplotlib == {matplotlib.__version__}')

pandas == 1.3.5
statsmodels == 0.13.2
matplotlib == 3.2.2


In [5]:
# ajustamos o formato de apresentação padrão dos gráficos
sns.set_theme(style="white", palette="pastel")

In [6]:
# habilitamos a visualização especial de dataframes disponível no
# Google Colaboratory, para facilitar a exploração dos dados.
from google.colab import data_table
data_table.enable_dataframe_formatter()

## Carga dos dados

In [7]:
path_dir_dados = Path('.') / NOME_DIRETORIO_DADOS

path_arq_ibcbr     = path_dir_dados / NOME_ARQUIVO_SERIE_IBCBR
path_arq_ibcbr_res = path_dir_dados / NOME_ARQUIVO_SERIE_IBCBR_RESIDUO
path_arq_dif       = path_dir_dados / NOME_ARQUIVO_SERIE_IBCBR_DIFERENCAS
path_arq_dif_res   = path_dir_dados / NOME_ARQUIVO_SERIE_IBCBR_DIFERENCAS_RESIDUO

In [8]:
def carregar_csv(path_csv:Path) -> pd.DataFrame:
  """
  Carrega os dados de um arquivo CSV em um dataframe de
  forma padronizada no escopo do projeto.
  path_csv : objeto Path que aponta para o arquivo.
  """
  if (path_csv is None) or (not path_csv.is_file()):
    raise ValueError("The given path object doesn't point to a valid csv file.")
  
  return pd.read_csv(path_csv,
                   sep=',',
                   parse_dates=True,
                   infer_datetime_format=True,
                   index_col=0,
                   decimal='.',
                   encoding='utf8')

In [9]:
# carregamos os arquivos de dados em dataframes, para iniciar a análise
df_ibcbr     = carregar_csv(path_arq_ibcbr)
df_ibcbr_res = carregar_csv(path_arq_ibcbr_res)
df_dif       = carregar_csv(path_arq_dif)
df_dif_res   = carregar_csv(path_arq_dif)

In [10]:
def verificar_propriedades(id:int, nome:str, dataframe:pd.DataFrame) -> None:
  """
  Imprime propriedades do dataframe nomeado para inspeção visual dos dados.
  id: inteiro que identifica o dataframe.
  nome: nome do dataframe
  dataframe: objeto do dataframe
  """
  print(f'[{id:03d}] Dataframe: {nome}')
  print('-' * 5)
  print(f'Shape: {dataframe.shape}')
  print('-' * 5)
  print(dataframe.head(4))
  print('')

In [11]:
# verificamos se os dataframes foram carregados corretamente.
series = {
  'IBC-BR': df_ibcbr,
  'IBC-BR (resíduo)': df_ibcbr_res,
  'Diferenças': df_dif,
  'Diferenças (resíduo)': df_dif_res
}

for idx,serie in enumerate(series.items()):
  verificar_propriedades(idx+1, serie[0], serie[1])

[001] Dataframe: IBC-BR
-----
Shape: (236, 1)
-----
             valor
data              
2003-01-01   96.15
2003-02-01   98.67
2003-03-01  103.41
2003-04-01  102.19

[002] Dataframe: IBC-BR (resíduo)
-----
Shape: (224, 1)
-----
               valor
data                
2003-07-01  0.988355
2003-08-01  0.972326
2003-09-01  1.007602
2003-10-01  1.009311

[003] Dataframe: Diferenças
-----
Shape: (235, 1)
-----
            valor
data             
2003-02-01   2.52
2003-03-01   4.74
2003-04-01  -1.22
2003-05-01  -1.89

[004] Dataframe: Diferenças (resíduo)
-----
Shape: (235, 1)
-----
            valor
data             
2003-02-01   2.52
2003-03-01   4.74
2003-04-01  -1.22
2003-05-01  -1.89



Neste ponto, temos todos os dados carregados em DataFrames pandas e prontos para análise.

## Análise estatística

Nossa primeira abordagem será a análise do z-score modificado das séries estacionárias, para tentar encontrar anomalias. O uso do z-score modificado no lugar do z-score padrão decorre (1) da premissa de que existem _outliars_ nos dados e (2) no fato que _outliars_ modificam a média e o desvio padrão, o que influencia o cálculo do z-score padrão. O z-score modificado usa a mediana e o desvio absoluto mediano (MAD), que são mais resilientes à _outliars_ nos dados.

Começamos definindo as funções auxiliares para obter as estatísticas desejadas.

In [12]:
def calcular_mad(dataframe:pd.DataFrame) -> float:
  """
  Calcula o desvio absoluto mediano (sigla MAD, do inglês 'median
  absolute deviation') dos dados de um dataframe.
  """
  mediana = np.median(dataframe)
  mad = np.median(np.abs(dataframe - mediana))

  return mad

def calcular_zscore_mod(x:float, mediana:float, mad:float, k:float=1.4826) -> float:
  """
  Calcula o z-score modificado de um elemento x, com base
  no elemento, em uma mediana e no MAD.
  x : elemento a considerar.
  mediana : mediana a considerar.
  MAD : desvio absoluto mediano a considerar.
  k : fator de correção (veja https://en.wikipedia.org/wiki/Median_absolute_deviation)
  retorna o z-score modificado calculado para o elemento.
  """
  assert (k > 0)
  assert (mad > 0)

  return (x - mediana) / (k * mad)

def calcular_zscore_mod(dataframe:pd.DataFrame, k:float=1.4826) -> pd.DataFrame:
  """
  Calcula o z-score modificado de todos os elementos de um
  dataframe. Não modifica o dataframe recebido.
  dataframe : dataframe para o qual se deseja calcular o z-score.
  k : fator de correção (veja https://en.wikipedia.org/wiki/Median_absolute_deviation)
  Retorna um objeto pandas.DataFrame com os valores dos z-scores
  dos elementos.
  """
  assert (k > 0)
  mediana = np.median(dataframe)
  mad = calcular_mad(dataframe)
  assert (mad > 0)
  return (dataframe - mediana) / (k * mad)

Depois, começamos a análise estatística do resíduo da série IBC-BR.

In [13]:
# imprimimos estatísticas do conjunto de dados só para auxiliar na análise
df_ibcbr_res.describe()

,valor
count,224.000000
mean,0.999796
std,0.017775
min,0.893994
25%,0.991451
50%,1.000503
75%,1.009582
max,1.052196


In [14]:
print(f'Mediana do resíduo do IBC-BR: {np.median(df_ibcbr_res)}')

Mediana do resíduo do IBC-BR: 1.0005033876381089


In [15]:
ibcbr_res_mad = calcular_mad(df_ibcbr_res)
print(f'MAD do resíduo do IBC-BR = {ibcbr_res_mad}.')

MAD do resíduo do IBC-BR = 0.009062644702768241.


In [16]:
# criamos uma cópia do dataframe e calculamos o z-score modificado
# de suas entradas
df_temp = df_ibcbr_res.copy()
df_temp['z-score mod'] = calcular_zscore_mod(df_temp)
display(df_temp.head())

,valor,z-score mod
data,,
2003-07-01,0.988355,-0.904159
2003-08-01,0.972326,-2.097129
2003-09-01,1.007602,0.528325
2003-10-01,1.009311,0.655498
2003-11-01,0.992433,-0.600664


In [17]:
# depois filtramos os dados em busca de anomalias,
# assumindo que qualquer coisa acima de 2.5 unidades
# de MAD é uma anomalia.

limiar_anomalia = 2.5
filtro_acima_lim_sup = df_temp['z-score mod'] > limiar_anomalia
filtro_abaixo_lim_inf = df_temp['z-score mod'] < -limiar_anomalia

df_anomalias = df_temp[ filtro_acima_lim_sup | filtro_abaixo_lim_inf ]

In [19]:
display(df_anomalias)

,valor,z-score mod
data,,
2008-07-01,1.034370,2.520569
2008-09-01,1.038398,2.820339
2008-12-01,0.955111,-3.378379
2009-01-01,0.959341,-3.063555
2009-02-01,0.960103,-3.006779
2020-01-01,1.046810,3.446424
2020-02-01,1.052196,3.847261
2020-04-01,0.893994,-7.927020
2020-05-01,0.904317,-7.158683


O processo parece ter detectado anomalias em 2008 e 2009 (potenciais efeitos da crise do mercado financeiro mundial de 2008) e de dezembro de 2019 até junho de 2020 (potenciais efeitos da pandemia de COVID-19).

In [18]:
# FIXME: Continuar daqui. Desenhar gráfico do IBC-BR vs. anomalias e realizar
# análise com base na série de diferenças. Depois usar soma cumulativa (CUMSUM)
# e comparar os resultados.